In [5]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_train.csv",index_col = 0)
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_test.csv")
print(df_train)

X = df_train.iloc[:,1:].values 
X_train = df_train.iloc[:1000,1:].values
X_test = df_train.iloc[1000:2000,1:].values
test_data = df_test.iloc[:,1:].values
ids = df_test.iloc[:,0].values
label = df_train.iloc[:,0].values
label_train = label[0:1000]
label_test = label[1000:2000]

k = 101

print(X)

#standardization

x = (X - np.mean(X,axis= 0 , keepdims = True))/255
x_train = (X_train - np.mean(X,axis = 0,keepdims = True))/255

class Data():
    def __init__(self,x,label):
        self.data = x
        self.label = label

row,col = X.shape
Train_data = [1]*row
for i in range (row):
    Train_data[0] = Data(x[i],label[i])

def Accuracy(y_pred , y_test):
    
    count = 0
    # y_pred = np.argmax(y_pred,axis = 1)
    # y_test = np.argmax(y_test,axis = 1)
    for i in range(y_pred.size):
        if (y_pred[i] - y_test[i] == 0):
            count+=1
    return count/y_test.size

       label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0          8       0       0       0       0       0       0       0       0   
1          4       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          8       0       0       0       0       0       0       0       0   
4          2       0       0       0       0       1       0       3       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
29995      3       0       0       0       0       0       0       0       0   
29996      4       0       0       0       0       0       0       0       0   
29997      9       0       0       0       0       0       0       0       0   
29998      4       0       0       0       0       1       0       0       0   
29999      6       0       0       0       1       0       0       0       0   

       pixel9  ...  pixel775  pixel776 

In [ ]:
losses , ks = [] , []
x_test = (X_test - np.mean(X,axis = 0, keepdims = True))/255
for k in range(5,int((x_test.shape[0])**0.5)):
    y_pred = knn(k,x_train,label_train,x_test)  
    loss = np.mean((label_train-y_pred)**2)
    ks.append(k)
    losses.append(loss)
plt.plot(ks,losses,c = 'c')
plt.xlabel('K ->')
plt.ylabel('Loss ->')
plt.show()

In [7]:
test_data = df_test.iloc[:,1:].values

x_test = (test_data - np.mean(X,axis = 0, keepdims = True))/255


def dist(x,x_test):
    a2 = x**2 
    b2 = x_test**2
    ab = 2 * x * x_test
    distance = np.sum(a2+b2-ab,axis = 1)

    return distance

def knn(k,x, label , x_test):
    row, col = x.shape
    y_pred = np.zeros(x_test.shape[0])
    for i in range(x_test.shape[0]):
        distance = np.array(dist(x,x_test[i]))
        #print(distance.shape)
        dist_sort = np.sort(distance)
        Nearest_k = dist_sort[:k]
        indices = np.array([],dtype = 'int32')
        #print(np.where(distance==dist_sort[0]))
        for j in range(k):
            indices = np.hstack([indices,np.where(distance==dist_sort[j])[0]])
        #print(indices)
        Nearest_label = label[indices]
        #print(Nearest_label)
        count = np.bincount(Nearest_label)  # puts stuff into bins whose index are values of Nearest_label  
        #print(count)
        y_pred[i] = np.argmax(count)
        #print(int(y_pred[i]))
        #np.argsort()
    return y_pred     
# print((x-x_test[0])[1:10,1:10])


y_pred = knn(k,x_train,label[:1000],x_test)

print (y_pred)

KeyboardInterrupt: ignored

In [ ]:
y_pred = y_pred.astype("int32")
res= pd.DataFrame([ids,y_pred[:10000]],index = ["Ids","Predicted label"],dtype= 'int32').T
res.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/KNN_Result.csv")
print(res)

        Ids  Predicted label
0     25672                0
1     59964                1
2     66230                2
3     50801                2
4     83307                3
...     ...              ...
9995  71009                0
9996  67472                6
9997  99226                8
9998  58519                8
9999  80458                4

[10000 rows x 2 columns]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')